# Neo4j code

In [7]:
import os
import requests
from neo4j import GraphDatabase
import subprocess
import time

### Neo4j connection
Docker should be running.
Installation and activation of Neo4j with Neosemantics plugin (as in https://neo4j.com/labs/neosemantics/installation/ Docker):
```powershell
docker run --name testneo4j -p 7474:7474 -p 7687:7687 -d `
      -v ${env:USERPROFILE}\neo4j\data:/data `
      -v ${env:USERPROFILE}\neo4j\logs:/logs `
      -v ${env:USERPROFILE}\neo4j\import:/var/lib/neo4j/import `
      -v ${env:USERPROFILE}\neo4j\plugins:/plugins `
      --env 'NEO4J_PLUGINS=[\"apoc\", \"n10s\"]' `
      --env NEO4J_AUTH=neo4j/neoforyou `
      neo4j:5.20.0 
```


In [10]:
# Connection setup
NEO4J_URI = "bolt://localhost:7687" # faster than http, 7687 is the default port for bolt, 7474 for http
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "neoforyou"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD)) # driver is the main object to interact with the database

# verify connection
try:
    driver.verify_connectivity()
    print("Neo4j connection verified :)")
except Exception as e:
    print("Neo4j connection failed :("
          "\nPlease check if docker is running and the correct command to start the Neo4j server")
    print(e)

# Sample TTL data
# prefix ex, "Person1" is type "Person" and has the property "name" with value "Juanito"...
sample_ttl = """
@prefix ex: <http://example.org/> .
ex:Person1 a ex:Person ; ex:name "Juanito" .
ex:Person2 a ex:Person ; ex:name "Eustaquia" .
"""

Neo4j connection verified :)


In [ ]:
# Only for the first time, to create the database
# Create unique URI constraint for RDF import
print("Creating unique URI constraint for RDF import and initializing graph configuration...")
with driver.session() as session:
    session.run("""
        CREATE CONSTRAINT n10s_unique_uri FOR (r:Resource) REQUIRE r.uri IS UNIQUE
    """)
    session.run("""
        CALL n10s.graphconfig.init();
    """)
print("Constraint created successfully.")

### Sample of TTL file into Neo4j

In [12]:
# Load TTL data into Neo4j using Neosemantics
print("Loading TTL data into Neo4j...")
with driver.session() as session:
    result = session.run("""
        CALL n10s.rdf.import.inline($data, 'Turtle')
    """, data=sample_ttl)

    summary = result.single() # get the first record
    if summary and summary["terminationStatus"] == "OK":
        print(f"TTL data loaded successfully :) Triples loaded: {summary['triplesLoaded']}")
    else:
        print(f"Failed to load TTL :( But I am sure you can solve the problem: /n {summary}")

Loading TTL data into Neo4j...
TTL data loaded successfully :) Triples loaded: 4


### Verification of loaded data

In [13]:
# Read data from Neo4j to check if it was loaded correctly
def query_neo4j(tx, query):
    result = tx.run(query)
    for record in result:
        print(record)

print("Running test query on Neo4j graph...")
with driver.session() as session:
    session.execute_read(query_neo4j, "MATCH (n) RETURN n LIMIT 5")

# Close connection
driver.close()

print("Neo4j TTL loading test completed :)")

Running test query on Neo4j graph...
<Record n=<Node element_id='4:df59c55f-6175-44fa-9c6b-fa85bf284ad0:0' labels=frozenset({'_GraphConfig'}) properties={'_classLabel': 'Class', '_handleRDFTypes': 0, '_subClassOfRel': 'SCO', '_handleMultival': 0, '_objectPropertyLabel': 'Relationship', '_rangeRel': 'RANGE', '_domainRel': 'DOMAIN', '_keepLangTag': False, '_keepCustomDataTypes': False, '_classNamePropName': 'name', '_handleVocabUris': 0, '_applyNeo4jNaming': False, '_relNamePropName': 'name', '_dataTypePropertyLabel': 'Property', '_subPropertyOfRel': 'SPO'}>>
<Record n=<Node element_id='4:df59c55f-6175-44fa-9c6b-fa85bf284ad0:1' labels=frozenset({'_NsPrefDef'}) properties={'ns0': 'http://example2.org/', 'ns1': 'http://example.org/'}>>
<Record n=<Node element_id='4:df59c55f-6175-44fa-9c6b-fa85bf284ad0:2' labels=frozenset({'Resource', 'ns0__Person'}) properties={'ns0__name': 'Naira', 'uri': 'http://example2.org/Person2'}>>
<Record n=<Node element_id='4:df59c55f-6175-44fa-9c6b-fa85bf284ad0:3